In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor

from sklearn import metrics as skm

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# from sklearn.tree import DecisionTreeClassifier, plot_tree

# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans, DBSCAN

# from keras.models import Sequential
# from keras.layers import Dense, Dropout

# from keras.callbacks import EarlyStopping
# from keras.wrappers.scikit_learn import KerasRegressor

# rdst = 815


Bad key "text.kerning_factor" on line 4 in
C:\Users\weisj\anaconda3\envs\dsi\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [8]:
pets = pd.read_csv('../data/pets.csv')

In [9]:
pets.fillna(0, inplace = True)

In [10]:
pets.columns = map(str.lower, pets.columns)

In [11]:
pets.drop(pets[pets['quantity'] > 3].index, inplace = True) 

In [12]:
pets = pd.get_dummies(pets, columns = ['breed1','breed2','color1', 'color2', 'color3','type', 'gender'], drop_first = True)

In [13]:
pets.isnull().sum().sum()

0

In [14]:
# Need to convert the adoption speed to numericals that reflect the timing
pets['adoptionspeed'].replace({
0:1,
1:7,
2:20,
3:60,
4:100,
}, inplace = True)
pets['adoptionspeed'][:5]

0    100
1    100
2     60
3    100
4     20
Name: adoptionspeed, dtype: int64

In [15]:
# Train-Test Split  - 7686x323 and 2563x323

X = pets.drop(columns = ['name', 'adoptionspeed', 'description', 'petid', 'rescuerid', 'state'])
y = pets['adoptionspeed']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = rdst)

print('X Train', X_train.shape)
print('X Test', X_test.shape)
print()
print('y Train', y_train.shape)
print('y Test', y_test.shape)

X Train (7686, 309)
X Test (2563, 309)

y Train (7686,)
y Test (2563,)


In [51]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [52]:
lr.score(X_train, y_train)

0.11740691837843163

In [53]:
lr.score(X_test,y_test)

-1.0903351367984938e+18

In [16]:
# Standard Scalar

ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test  = ss.transform(X_test)
print('Z Train', Z_train.shape)
print('Z Test', Z_test.shape)


Z Train (7686, 309)
Z Test (2563, 309)


In [18]:
knn = KNeighborsRegressor(n_neighbors = 5)
cross_val_score(knn, Z_train, y_train, cv =5).mean()

NameError: name 'Z' is not defined

In [19]:
# Fit KNN
knn.fit(Z_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [20]:
print("Train Score: ", knn.score(Z_train, y_train))
print('Test Score: ',  knn.score(Z_test, y_test))
print('Cross-Val: ',   cross_val_score(knn,Z_train, y_train).mean())

Train Score:  0.28771112701360313
Test Score:  -0.04591662753333381
Cross-Val:  -0.07582280714709588


In [21]:
dt = DecisionTreeRegressor(random_state = rdst)
dt.fit(X_train, y_train)
print(f'Score on training set: {dt.score(X_train, y_train)}')
print(f'Score on testing set: {dt.score(X_test, y_test)}')

Score on training set: 0.9798150781475911
Score on testing set: -0.6238822882903479


In [22]:
grid = GridSearchCV(DecisionTreeRegressor(),
                   param_grid = {'max_depth' : [2,3,4,5,6,7,8,9],
                                 'min_samples_split' : [5, 10, 15, 20, 25, 30],
                                 'min_samples_leaf' : [2, 3, 4, 5, 6, 7, 8, 9]},
                                cv = 5,
                                 verbose = 2) # to see it work
                                

In [23]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.1s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_spl

[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_spli

[CV]  max_depth=2, min_samples_leaf=5, min_samples_split=30, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_spli

...

[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples

[CV]  max_depth=9, min_samples_leaf=7, min_samples_split=15, total=   0.1s
[CV] max_depth=9, min_samples_leaf=7, min_samples_split=15 ...........
[CV]  max_depth=9, min_samples_leaf=7, min_samples_split=15, total=   0.1s
[CV] max_depth=9, min_samples_leaf=7, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=7, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=7, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=7, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=7, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=7, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=7, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=7, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=7, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=7, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=7, min_samples

[CV]  max_depth=9, min_samples_leaf=9, min_samples_split=15, total=   0.1s
[CV] max_depth=9, min_samples_leaf=9, min_samples_split=15 ...........
[CV]  max_depth=9, min_samples_leaf=9, min_samples_split=15, total=   0.1s
[CV] max_depth=9, min_samples_leaf=9, min_samples_split=15 ...........
[CV]  max_depth=9, min_samples_leaf=9, min_samples_split=15, total=   0.1s
[CV] max_depth=9, min_samples_leaf=9, min_samples_split=15 ...........
[CV]  max_depth=9, min_samples_leaf=9, min_samples_split=15, total=   0.1s
[CV] max_depth=9, min_samples_leaf=9, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=9, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=9, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=9, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=9, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=9, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=9, min_samples

[Parallel(n_jobs=1)]: Done 1920 out of 1920 | elapsed:  2.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9],
                    

In [24]:
dec_tree = grid.best_estimator_
dec_tree

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=30,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [25]:
print('Train Score: ', grid.score(X_train, y_train))
print('Test Score: ', grid.score(X_test, y_test))
print('Cross Val: ', cross_val_score(dec_tree, X_train, y_train).mean())

Train Score:  0.1714087281962856
Test Score:  0.123552392773709
Cross Val:  0.13595600100728017


Yikes, those are some bad scores!

## Neural Network

In [40]:
Z_train.shape[1]

309

In [49]:
def model_fn(layer_one_neurons = 32, layer_two_neurons = 32, layer_one_dropout = .5):
    model = Sequential()
    model.add(Dense(layer_one_neurons, 
                    activation = 'relu', 
                    input_shape = (Z_train.shape[1],)))
    model.add(Dropout(layer_one_dropout))
    model.add(Dense(layer_two_neurons, activation = 'relu'))
    model.add(Dense(1, activation = None))
    model.compile(loss = 'mse', optimizer = 'adam')
    return model

nn = KerasRegressor(build_fn = model_fn, batch_size = 512, verbose = 1)
nn_params = {
    'epochs' : [100,500],
    'layer_one_neurons' : [500, 1_000, 3_000],
    'layer_one_dropout' : [.5],
    'layer_two_neurons'  : [64, 128]}
gs = GridSearchCV(nn, param_grid = nn_params, cv = 5)
gs.fit(Z_train, y_train)

print(gs.best_score_)
gs.best_params_


Epoch 1/100
6148/6148 [==============================] - 0s 41us/step - loss: 3375.8227
Epoch 2/100
6148/6148 [==============================] - 0s 40us/step - loss: 2696.0207
Epoch 3/100
6148/6148 [==============================] - 0s 40us/step - loss: 2067.1330
Epoch 4/100
6148/6148 [==============================] - 0s 42us/step - loss: 1614.5494
Epoch 5/100
6148/6148 [==============================] - 0s 41us/step - loss: 1401.0059
Epoch 6/100
6148/6148 [==============================] - 0s 41us/step - loss: 1315.9875
Epoch 7/100
6148/6148 [==============================] - 0s 51us/step - loss: 1283.9236
Epoch 8/100
6148/6148 [==============================] - 0s 39us/step - loss: 1256.7356
Epoch 9/100
6148/6148 [==============================] - 0s 42us/step - loss: 1242.7404
Epoch 10/100
6148/6148 [==============================] - 0s 42us/step - loss: 1230.4042
Epoch 11/100
6148/6148 [==============================] - 0s 42us/step - loss: 1215.4941
Epoch 12/100
6148/6148 [======

6148/6148 [==============================] - 0s 41us/step - loss: 1054.1446
Epoch 94/100
6148/6148 [==============================] - 0s 42us/step - loss: 1046.9914
Epoch 95/100
6148/6148 [==============================] - 0s 39us/step - loss: 1049.1148
Epoch 96/100
6148/6148 [==============================] - 0s 39us/step - loss: 1059.6717
Epoch 97/100
6148/6148 [==============================] - 0s 43us/step - loss: 1053.0715
Epoch 98/100
6148/6148 [==============================] - 0s 44us/step - loss: 1057.0132
Epoch 99/100
6148/6148 [==============================] - 0s 44us/step - loss: 1052.8625
Epoch 100/100
1538/1538 [==============================] - 0s 22us/step
Epoch 1/100
6149/6149 [==============================] - 0s 38us/step - loss: 3326.8793
Epoch 2/100
6149/6149 [==============================] - 0s 36us/step - loss: 2628.0105
Epoch 3/100
6149/6149 [==============================] - 0s 40us/step - loss: 2003.3575
Epoch 4/100
6149/6149 [==============================]

6149/6149 [==============================] - 0s 56us/step - loss: 1038.3061
Epoch 85/100
6149/6149 [==============================] - 0s 57us/step - loss: 1048.4182
Epoch 86/100
6149/6149 [==============================] - 0s 54us/step - loss: 1047.7383
Epoch 87/100
6149/6149 [==============================] - 0s 46us/step - loss: 1043.0906
Epoch 88/100
6149/6149 [==============================] - 0s 51us/step - loss: 1049.3987
Epoch 89/100
6149/6149 [==============================] - 0s 58us/step - loss: 1040.2794
Epoch 90/100
6149/6149 [==============================] - 0s 50us/step - loss: 1045.5701
Epoch 91/100
6149/6149 [==============================] - 0s 50us/step - loss: 1038.4822
Epoch 92/100
6149/6149 [==============================] - 0s 55us/step - loss: 1041.9314
Epoch 93/100
6149/6149 [==============================] - 0s 52us/step - loss: 1024.8359
Epoch 94/100
6149/6149 [==============================] - 0s 47us/step - loss: 1027.0956
Epoch 95/100
6149/6149 [==========

6149/6149 [==============================] - 0s 43us/step - loss: 1049.4663
Epoch 76/100
6149/6149 [==============================] - 0s 39us/step - loss: 1060.1554
Epoch 77/100
6149/6149 [==============================] - 0s 39us/step - loss: 1052.1157
Epoch 78/100
6149/6149 [==============================] - 0s 43us/step - loss: 1044.5128
Epoch 79/100
6149/6149 [==============================] - 0s 41us/step - loss: 1047.2286
Epoch 80/100
6149/6149 [==============================] - 0s 43us/step - loss: 1039.0150
Epoch 81/100
6149/6149 [==============================] - 0s 45us/step - loss: 1051.7483
Epoch 82/100
6149/6149 [==============================] - 0s 40us/step - loss: 1044.2260
Epoch 83/100
6149/6149 [==============================] - 0s 40us/step - loss: 1042.5300
Epoch 84/100
6149/6149 [==============================] - 0s 38us/step - loss: 1039.5145
Epoch 85/100
6149/6149 [==============================] - 0s 40us/step - loss: 1042.5721
Epoch 86/100
6149/6149 [==========

6149/6149 [==============================] - 0s 42us/step - loss: 1096.0503
Epoch 67/100
6149/6149 [==============================] - 0s 39us/step - loss: 1081.0607
Epoch 68/100
6149/6149 [==============================] - 0s 42us/step - loss: 1100.0605
Epoch 69/100
6149/6149 [==============================] - 0s 41us/step - loss: 1090.2820
Epoch 70/100
6149/6149 [==============================] - 0s 41us/step - loss: 1090.7194
Epoch 71/100
6149/6149 [==============================] - 0s 43us/step - loss: 1076.4587
Epoch 72/100
6149/6149 [==============================] - 0s 42us/step - loss: 1079.9741
Epoch 73/100
6149/6149 [==============================] - 0s 43us/step - loss: 1079.4761
Epoch 74/100
6149/6149 [==============================] - 0s 41us/step - loss: 1083.4600
Epoch 75/100
6149/6149 [==============================] - 0s 41us/step - loss: 1070.8812
Epoch 76/100
6149/6149 [==============================] - 0s 41us/step - loss: 1082.7243
Epoch 77/100
6149/6149 [==========

...

6149/6149 [==============================] - 10s 2ms/step - loss: 738.8133
Epoch 262/500
6149/6149 [==============================] - 9s 2ms/step - loss: 735.2575
Epoch 263/500
6149/6149 [==============================] - 11s 2ms/step - loss: 719.4178
Epoch 264/500
6149/6149 [==============================] - 9s 1ms/step - loss: 714.9347
Epoch 265/500
6149/6149 [==============================] - 10s 2ms/step - loss: 723.7775
Epoch 266/500
6149/6149 [==============================] - 8s 1ms/step - loss: 738.9119
Epoch 267/500
6149/6149 [==============================] - 9s 1ms/step - loss: 724.3261
Epoch 268/500
6149/6149 [==============================] - 11s 2ms/step - loss: 720.0087
Epoch 269/500
6149/6149 [==============================] - 8s 1ms/step - loss: 708.5002
Epoch 270/500
6149/6149 [==============================] - 10s 2ms/step - loss: 721.5593
Epoch 271/500
6149/6149 [==============================] - 9s 2ms/step - loss: 727.9154
Epoch 272/500
6149/6149 [================

6149/6149 [==============================] - 8s 1ms/step - loss: 671.1824
Epoch 355/500
6149/6149 [==============================] - 10s 2ms/step - loss: 682.7042
Epoch 356/500
6149/6149 [==============================] - 8s 1ms/step - loss: 674.5178
Epoch 357/500
6149/6149 [==============================] - 9s 1ms/step - loss: 674.1454
Epoch 358/500
6149/6149 [==============================] - 8s 1ms/step - loss: 702.0394
Epoch 359/500
6149/6149 [==============================] - 9s 1ms/step - loss: 664.5757
Epoch 360/500
6149/6149 [==============================] - 8s 1ms/step - loss: 668.1241
Epoch 361/500
6149/6149 [==============================] - 9s 1ms/step - loss: 676.0510
Epoch 362/500
6149/6149 [==============================] - 8s 1ms/step - loss: 667.4963
Epoch 363/500
6149/6149 [==============================] - 9s 1ms/step - loss: 666.7900
Epoch 364/500
6149/6149 [==============================] - 8s 1ms/step - loss: 687.2677
Epoch 365/500
6149/6149 [====================

6149/6149 [==============================] - 2s 266us/step - loss: 628.0144
Epoch 446/500
6149/6149 [==============================] - 2s 261us/step - loss: 701.2748
Epoch 447/500
6149/6149 [==============================] - 2s 266us/step - loss: 648.5553
Epoch 448/500
6149/6149 [==============================] - 2s 264us/step - loss: 632.4739
Epoch 449/500
6149/6149 [==============================] - 2s 257us/step - loss: 614.8288
Epoch 450/500
6149/6149 [==============================] - 2s 268us/step - loss: 608.2073
Epoch 451/500
6149/6149 [==============================] - 2s 261us/step - loss: 642.5367
Epoch 452/500
6149/6149 [==============================] - 2s 274us/step - loss: 630.1085
Epoch 453/500
6149/6149 [==============================] - 2s 265us/step - loss: 660.3701
Epoch 454/500
6149/6149 [==============================] - 2s 282us/step - loss: 631.7526
Epoch 455/500
6149/6149 [==============================] - 2s 266us/step - loss: 622.8539
Epoch 456/500
6149/6149 

7686/7686 [==============================] - 0s 45us/step - loss: 1138.9844
Epoch 37/100
7686/7686 [==============================] - 0s 44us/step - loss: 1132.6264
Epoch 38/100
7686/7686 [==============================] - 0s 44us/step - loss: 1144.7941
Epoch 39/100
7686/7686 [==============================] - 0s 45us/step - loss: 1128.6127
Epoch 40/100
7686/7686 [==============================] - 0s 46us/step - loss: 1127.8506
Epoch 41/100
7686/7686 [==============================] - 0s 45us/step - loss: 1131.1135
Epoch 42/100
7686/7686 [==============================] - 0s 45us/step - loss: 1135.9219
Epoch 43/100
7686/7686 [==============================] - 0s 46us/step - loss: 1131.4602
Epoch 44/100
7686/7686 [==============================] - 0s 45us/step - loss: 1126.5156
Epoch 45/100
7686/7686 [==============================] - 0s 50us/step - loss: 1117.4588 0s - los
Epoch 46/100
7686/7686 [==============================] - 0s 48us/step - loss: 1114.5738
Epoch 47/100
7686/7686 [=

{'epochs': 100,
 'layer_one_dropout': 0.5,
 'layer_one_neurons': 500,
 'layer_two_neurons': 128}

In [45]:
kr_best = gs.best_score_
gs.best_params_

{'epochs': 100,
 'layer_one_dropout': 0.5,
 'layer_one_neurons': 500,
 'layer_two_neurons': 64}

In [48]:
np.sqrt(-kr_best)

36.16335694120427